In [ ]:
%load_ext autoreload 
%autoreload 2
from utils import load_data, load_excel_sheets, plot_distribution_bar
from config import PARSED_DATA_PATH, CANDIDATE_CVS_PATH, REVERSE_MATCHING_PATH, JOB_LINKS
from gender_analysis import get_category_distribution, get_skill_distribution_by_gender, get_skill_gender_share,plot_gender_bias_skills_bar, get_skilltype_gender_share
import polars as pl 

In [ ]:
cv_skills = load_data(PARSED_DATA_PATH)
dfs = load_excel_sheets(REVERSE_MATCHING_PATH, ["Candidates"])

In [ ]:
df_skills_with_gender = cv_skills.join(dfs["Candidates"].select(["CANDIDATE_ID", "Gender"]), 
                    left_on="CANDIDATE_ID", 
                    right_on="CANDIDATE_ID", 
                    how="inner")

In [ ]:
df_skills_with_gender

In [ ]:
gender_counts_df = get_category_distribution(df_skills_with_gender, "Gender")
gender_counts_df

In [ ]:
plot_distribution_bar(
    gender_counts_df,
    x_col="Gender",
    y_col="count",
    x_label="Gender",
    y_label="Number of Candidates",
    title="Candidate Distribution by Gender"
)

In [ ]:
df_category_with_gender = get_skilltype_gender_share(df_skills_with_gender)
df_category_with_gender

In [ ]:
plot_gender_bias_skills_bar(df_category_with_gender,"Skill_Type","perc_male","perc_female","perc_diff")

In [ ]:
skill_distribution_df = get_skill_distribution_by_gender(df_skills_with_gender)
skill_distribution_df

In [ ]:
df_gender_bias = get_skill_gender_share(df_skills_with_gender)
df_gender_bias

In [ ]:
df_gender_bias.sort(pl.col("bias_strength"), descending=True).head(20)

In [ ]:
plot_gender_bias_skills_bar(df_gender_bias, "Skill","perc_male","perc_female","bias_strength", top_n=20)

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("LabHC/bias_in_bios", split='train')
df_train = train_dataset.to_pandas()
print(df_train.head())

In [ ]:
print(df_train.columns)

In [ ]:
df = df_gender_bias.sort("bias_strength", descending=True).head(30)
df

In [ ]:
job_df = df.filter(pl.col("Skill_Type") == "Job_title")
job_df.drop(["Skill_right","Skill_Type_right"])

In [ ]:
job_links = pl.Series(JOB_LINKS)
job_df.with_columns(job_links.alias("job_links"))